Evan Edelstein
EN.605.645.82.SP26

# Module 2 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else...
4. You must follow the Programming Requirments for this course.

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

<div style="color: white; background: #C83F49; margin:20px; padding: 20px;">
    <strong>Academic Integrity and Copyright</strong>
    <p>You are not permitted to consult outside sources (Stackoverflow, YouTube, ChatGPT, etc.) or use "code assistance" (Co-Pilot, etc) to complete this assignment. By submitting this assignment for grading, you certify that the submission is 100% your own work, based on course materials, group interactions, instructor guidance. You agree to comply by the requirements set forth in the Syllabus, including, by reference, the JHU KSAS/WSE Graduate Academic Misconduct Policy.</p>
    <p>Sharing this assignment either directly (e.g., email, github, homework site) or indirectly (e.g., ChatGPT, machine learning platform) is a violation of the copyright. Additionally, all such sharing is a violation the Graduate Academic Misconduct Policy (facilitating academic dishonesty is itself academic dishonesty), even after you graduate.</p>
    <p>If you have questions or if you're unsure about the policy, ask via Canvas Inbox. In this case, being forgiven is <strong>not</strong> easier than getting permission and ignorance is not an excuse.</p>
    <p>This assignment is copyright (&copy Johns Hopkins University &amp; Stephyn G. W. Butcher). All rights reserved.</p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
⛰       hills      5
🐊       swamp      7
🌋       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾⛰⛰⛰🌾🌾🌾
🌾🌾⛰⛰🌾🌾🌾
🌾🌾⛰🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
    ["🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🐊", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "⛰", "⛰"],
    ["🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "⛰", "🌾", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "🌋", "🌋", "⛰"],
    ["🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "⛰", "⛰", "🌾", "🌾", "🐊", "🐊", "🐊", "🐊", "🌾", "🌾", "🌋", "🌾", "🌾", "🌾", "⛰", "🌾", "🌾", "⛰", "⛰", "🌋", "🌾"],
    ["🌾", "🌾", "🌾", "🌾", "⛰", "⛰", "🌋", "⛰", "⛰", "🐊", "🐊", "🐊", "🐊", "🐊", "🌾", "🌋", "🌋", "🌋", "🌋", "🌾", "⛰", "🌾", "🌾", "🌾", "⛰", "🌋", "🌾"],
    ["🌾", "🌾", "🌋", "⛰", "⛰", "🌋", "🌋", "⛰", "⛰", "🐊", "🐊", "🐊", "🐊", "🐊", "🌋", "🌋", "🌲", "🌋", "🌋", "🌋", "⛰", "⛰", "🌾", "🌾", "⛰", "⛰", "🌾"],
    ["🌲", "🌾", "🌋", "🌋", "🌋", "🌋", "⛰", "⛰", "⛰", "🐊", "🐊", "🐊", "🌾", "🌾", "🌾", "🌋", "🌲", "🌲", "🌋", "🌋", "⛰", "⛰", "🌾", "⛰", "⛰", "🌾", "🌾"],
    ["🌲", "🌾", "🌲", "🌋", "🌋", "⛰", "⛰", "⛰", "🌾", "🌾", "🐊", "🌾", "🌾", "🌾", "🌾", "🌲", "🌲", "🌲", "🌲", "🌋", "🌋", "⛰", "⛰", "⛰", "🌾", "🌾", "🌾"],
    ["🌲", "🌲", "🌲", "🌋", "🌲", "⛰", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "⛰", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌋", "🌋", "⛰", "⛰", "🌾", "🌾", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌲", "🌾", "🌾", "🌾", "🌾", "⛰", "⛰", "⛰", "⛰", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌋", "⛰", "⛰", "🌾", "🌾", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "⛰", "🌋", "🌋", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌋", "🌋", "⛰", "🌾", "🌾", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "🌋", "🌋", "🌋", "⛰", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌋", "🌾", "🌾", "⛰", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🐊", "🌾", "⛰", "🌾", "🌾", "🌋", "🌋", "⛰", "⛰", "🌾", "⛰", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌋", "🌾", "🌋", "⛰", "🌾"],
    ["🌲", "🌲", "🌲", "🐊", "🐊", "🐊", "🌋", "🌾", "⛰", "🌋", "🌋", "🌾", "🌾", "🌾", "⛰", "🌋", "🌲", "🌲", "🌲", "🌲", "🌲", "🌋", "🌋", "🌾", "🌋", "🌋", "⛰"],
    ["🌲", "🌲", "🌲", "🐊", "🐊", "🐊", "🌋", "⛰", "⛰", "🌋", "⛰", "🌾", "🐊", "🌾", "⛰", "🌋", "🌲", "🌲", "🌲", "🌾", "🌾", "🌋", "🌾", "🌾", "🌋", "🌋", "⛰"],
    ["🌲", "🌲", "🌲", "🌲", "🐊", "🐊", "🌋", "🌋", "🌋", "🌋", "🌾", "🌾", "🐊", "🌾", "⛰", "🌋", "🌋", "🌲", "🌾", "🌾", "🌋", "🌾", "🌾", "🌾", "⛰", "🌋", "⛰"],
    ["🌾", "🌲", "🌲", "🌲", "🌲", "🐊", "🐊", "🌋", "🌋", "🌾", "🌾", "🌾", "🐊", "🐊", "🌾", "⛰", "🌋", "🌲", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "⛰", "🌋", "⛰"],
    ["🌾", "🌾", "🌲", "🌲", "🌲", "🐊", "🐊", "🌋", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🐊", "⛰", "🌋", "🌋", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "🌋", "⛰", "⛰"],
    ["🌾", "🌾", "🌋", "🌲", "🌲", "🌾", "🐊", "🐊", "🐊", "🌾", "🌾", "🐊", "🌾", "🐊", "🐊", "🌾", "🌾", "🌋", "⛰", "🌾", "🌾", "🌾", "⛰", "🌋", "🌋", "⛰", "🌾"],
    ["🌾", "🌋", "🌋", "🌲", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🌾", "🌋", "⛰", "🌾", "🌾", "🌾", "⛰", "🌋", "🌾", "⛰", "🌾"],
    ["🌾", "🌋", "🌋", "🌾", "🌾", "🌾", "🌾", "🐊", "🌾", "🌾", "⛰", "🌾", "🌾", "🌾", "🌾", "🌾", "🌋", "🌋", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "🌋", "⛰", "🌾"],
    ["🌾", "🌋", "⛰", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "⛰", "🌋", "⛰", "⛰", "🌾", "🌾", "⛰", "🌋", "🌾", "🌾", "🌾", "🐊", "🐊", "🌾", "⛰", "🌋", "🌋", "🌾"],
    ["🌾", "🌋", "⛰", "⛰", "⛰", "🌾", "🌾", "🌾", "⛰", "⛰", "🌋", "🌋", "🌋", "⛰", "⛰", "🌋", "🌋", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🌾", "⛰", "🌋", "⛰"],
    ["🌾", "🌋", "⛰", "⛰", "🌋", "⛰", "🌾", "⛰", "⛰", "⛰", "🌋", "🌋", "⛰", "🌾", "🌋", "🌋", "🌾", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🐊", "⛰", "🌋", "⛰"],
    ["🌾", "🌋", "🌋", "🌋", "🌋", "🌋", "⛰", "⛰", "⛰", "🌾", "⛰", "⛰", "🌾", "🌾", "⛰", "⛰", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🐊", "🐊", "🐊", "🐊", "⛰"],
    ["🌾", "🌋", "🌋", "🌋", "🌋", "⛰", "🌾", "⛰", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🐊", "🐊", "🐊", "🐊", "🌾"],
    ["🌾", "🌾", "⛰", "⛰", "⛰", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🐊", "🐊", "🐊", "🐊", "🌾"],
    ["🌾", "🌾", "⛰", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🐊", "🐊", "🐊", "🐊", "🐊", "🐊", "🐊", "🌾"],
]

Note: emojis are not fixed width so here is a function that will print out a List of Lists of emojis in an HTML table.

In [2]:
from IPython.display import display_html


def display_emoji_grid(emoji_grid):
    """
    Display a List of Lists of emojis in a perfect grid (table) in a Jupyter Notebook.

    Parameters:
    emoji_grid (list of list of str): A 2D list containing emojis to display in a grid.
    """
    # Create HTML table
    html = '<table style="border-collapse: collapse;">'

    for row in emoji_grid:
        html += "<tr>"
        for emoji in row:
            html += f'<td style="border: none; padding: 0px; text-align: center; font-size: 1em;">{emoji}</td>'
        html += "</tr>"

    html += "</table>"

    # Display the HTML table
    display_html(html, raw=True)


#

In [3]:
display_emoji_grid(full_world)

🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,🌾,🌾,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,🌾,🌾,🌾,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,🌾,🌾,⛰,⛰,🌾
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,🌾,⛰,⛰,🌾,🌾
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,🌾,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,🌾


## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [4]:
small_world = [
    ["🌾", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲"],
    ["🌾", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲"],
    ["🌾", "🌲", "🌲", "🌲", "🌲", "🌲", "🌲"],
    ["🌾", "🌾", "🌾", "🌾", "🌾", "🌾", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌾"],
    ["🌲", "🌲", "🌲", "🌲", "🌲", "🌲", "🌾"],
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [5]:
MOVES = [(0, -1), (1, 0), (0, 1), (-1, 0)]

## Costs

We can encode the costs described above in a `Dict`:

In [6]:
COSTS = {"🌾": 1, "🌲": 3, "⛰": 5, "🐊": 7}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

* Do not print raw data structures
* Do not insert unneeded/unrequested spaces
* Use the provided emojis!
* Use the provided `display_emoji_grid` function.

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [7]:
from typing import List, Tuple, Dict, Callable, Set
from copy import deepcopy

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="priority_enqueue"></a>
## priority_enqueue

`priority_enqueue` Add a node with a given priority to a list-backed min-priority queue. After inserting the node into the queue, the queue is sorted by priority (lowest -> highest) to ensure the ordering is maintained. This function mutates the priority queue. **Used by**: [a_star_search](#a_star_search)

* **pq** List[Tuple[Tuple[int, int], float]]: priority queue of coordinates, each element is a tuple containing the (x,y) coordinate and a floating point priority value
* **node** Tuple[int, int]: a location in the world map, `(x, y)`.
* **priority** Float: priority value to order queue by (*lower value indicates higher priority*)

**returns** None:

In [8]:
def priority_enqueue(pq: List[Tuple[Tuple[int, int], float]], node: Tuple[int, int], priority: float):
    pq.append((node, priority))  # insert node into pq
    pq.sort(key=lambda x: x[1])  # mutates pq (pass-by-obj-ref)
    return

In [9]:
# test 1: pq maintains sorted order
pq = []
priority_enqueue(pq, (2, 2), 2)
priority_enqueue(pq, (0, 0), 0)
priority_enqueue(pq, (1, 1), 1)
assert pq == [((0, 0), 0), ((1, 1), 1), ((2, 2), 2)]

# test 2: ties are resolved by insert precedence
pq = []
priority_enqueue(pq, (2, 2), 0)
priority_enqueue(pq, (0, 0), 0)
priority_enqueue(pq, (1, 1), 0)
assert pq == [((2, 2), 0), ((0, 0), 0), ((1, 1), 0)]


# test set 3: abnormal int values
pq = []
priority_enqueue(pq, (2, 2), 0.42)
priority_enqueue(pq, (0, 0), -1)
priority_enqueue(pq, (1, 1), 0x199)

assert pq == [((0, 0), -1), ((2, 2), 0.42), ((1, 1), 409)]


<a id="priority_dequeue"></a>
## priority_dequeue

`priority_dequeue` Retrieve the node in the priority queue with the lowest priority value. This function mutates the priority queue. **Used by**: [a_star_search](#a_star_search)

* **pq** List[Tuple[Tuple[int, int], float]]: priority queue of coordinates, each element is a tuple containing the (x,y) coordinate and a floating point priority value

**returns** (int, int): coordinate of next priority in queue, `(x, y)`.

In [10]:
def priority_dequeue(pq: List[Tuple[Tuple[int, int], float]]):
    return pq.pop(0)[0]

In [11]:
# test 1: dequeue returns lowest priority node
pq = []
priority_enqueue(pq, (2, 2), 2)
priority_enqueue(pq, (0, 0), 0)
priority_enqueue(pq, (1, 1), 1)

result = priority_dequeue(pq)
assert result == (0, 0)

# test 2: ties are resolved by insert precedence
pq = []
priority_enqueue(pq, (2, 2), 0)
priority_enqueue(pq, (0, 0), 0)
priority_enqueue(pq, (1, 1), 0)

result = priority_dequeue(pq)
assert result == (2, 2)


# test set 3: abnormal int values
pq = []
priority_enqueue(pq, (2, 2), 0.42)
priority_enqueue(pq, (0, 0), -1)
priority_enqueue(pq, (1, 1), 0x199)

result = priority_dequeue(pq)
assert result == (0, 0)

<a id="get_neighbors"></a>
## get_neighbors

`get_neighbors` Generate the valid transition states from a given position. A 2d list of terrain information is provided to check for the world boundary and inaccessible locations, such as mountain spaces. A list of valid actions, in terms of x,y offsets is supplied and tested to generate the list of valid spaces. This function assumes all rows in the world have the same number of elements. **Used by**: [a_star_search](#a_star_search)

* **world** List[List[str]]: the actual context for the navigation problem.
* **node** Tuple[int, int]: the current location of the bot, `(x, y)`.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.

**returns** List[Tuple[int, int]]: successor location list

In [12]:
def get_neighbors(world: List[List[str]], node: Tuple[int, int], moves: List[Tuple[int, int]]) -> List[Tuple[int, int]]:
    max_x = len(world[0]) - 1  # Assumes all rows are the same size
    max_y = len(world) - 1
    valid_moves: List[Tuple[int, int]] = []

    for dx, dy in moves:
        x: int = node[0] + dx
        y: int = node[1] + dy
        # Bounds check + non-accessible location
        if 0 <= x <= max_x and 0 <= y <= max_y and world[y][x] != "🌋":
            valid_moves.append((x, y))

    return valid_moves

In [13]:
# Test 1 - boundary
world = small_world
moves = MOVES
ns = get_neighbors(world, (0, 0), moves)

assert ns == [(1, 0), (0, 1)]

# Test 2 - mountains blocking spaces
world = full_world
moves = MOVES
ns = get_neighbors(world, (5, 3), moves)
assert ns == [(5, 2), (4, 3)]

# Test 3 - mountains blocking all possible moves
world = full_world
moves = MOVES
ns = get_neighbors(world, (4, 23), moves)

assert ns == []

<a id="heuristic"></a>
## heuristic

`heuristic` Compute the Euclidean distance between two points. This is as an admissible heuristic function due to the triangle inequality. The heuristic is used to guide the selection of nodes to search, prioritizing nodes that close the distance to the goal.  **Used by**: [a_star_search](#a_star_search)

* **start** Tuple[int, int]: the current location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.

**returns** Float: Euclidean distance between start and goal

In [14]:
from math import sqrt

In [ ]:
def heuristic(start: Tuple[int, int], goal: Tuple[int, int]) -> float:
    distance = sqrt((goal[0] - start[0])**2 + (goal[1] - start[1])**2) # √((x2-x1)^2 + (y2-y1)^2))
    return distance

In [16]:
# assertions/unit tests

# test 1: validate equation
start = (3, 4)
goal = (3, 5)
dist = heuristic(start, goal)
assert dist == 1.0

# test 2: distance gets smaller as we approach the goal
start1 = (3, 4)
goal = (7, 16)
dist1 = heuristic(start, goal)

start2 = (6, 10)
dist2 = heuristic(start2, goal)
assert dist2 < dist1

# test 3: if goal == start the distance is zero
start = (12, 12)
goal = (12, 12)
dist = heuristic(start, goal)
assert dist == 0.0

# test 4a: distance to the left and up are still positive
start = (3, 6)
goal = (3, 5)
dist = heuristic(start, goal)
assert dist == 1.0

# test 4b: distance to the left and up are still positive
start = (4, 5)
goal = (3, 5)
dist = heuristic(start, goal)
assert dist == 1.0


<a id="a_star_search"></a>
## a_star_search

*add documentation/description of the algorithm here; connect it to the theory!*

*`a_star_search` Given a terrain map, starting and goal positions, a movement model, a cost associated with traversing each terrain type and a function to compute a heuristic, return a list of movements to reach the goal position from the starting position. The terrain map is specified as a 2d nested array, where each cell represents a type of terrain. The map is indexed in a similar style of matrices, that is, position (x,y) refers to the position x cells to the right and y cells down. Note that the map and positions are zero-indexed. Movements are encoded as offsets from a position in the map, we can encode movement in the four cardinal directions from a point (x,y) as: West: (x-1, 0) | East: (x+1,y) | North: (x, y-1) | South: (x,y+1). The movement logic and transition logic is encapsulated in the [get_neighbors](#get_neighbors) function. To store the nodes in sorted order (based on path cost and heuristic), a priority queue data structure is used. The functions [priority_enqueue](#priority_enqueue) and [priority_dequeue](#priority_dequeue) allow for the insertion and retrieval of nodes with the correct ordering.*

*The algorithm begins by initializing a frontier to store the nodes to travel too in order of priority using a priority queue, a set to maintain a list of explored nodes, a dictionary to store the parent-child linkage of each node and a dictionary to store the lowest path cost to a node. The starting node is added to the frontier, and path cost dictionary. While there are nodes on the frontier the next (lowest) priority node is retrieved. If it has the same coordinates as the goal, the path is retraced using the path dictionary and a list of steps from the start to goal is returned. If the node is not at the goal, its valid neighboring nodes are retrieved, and their path cost is calculated. If the neighbor has not been explored and its cost is lower than an alternative route, it is added to the frontier and path dictionary, and the path cost dictionary is updated. Once the neighbors have been expanded, the current node is added to the explore list, and the algorithm continues with the next node on the frontier.*

*The core of the A* algorithm is the use of a linear combination of path-cost and heuristic value as a priority for each node in the frontier to guide the search towards paths that minimize cost and close the distance to the goal. Mathematically, we express the priority as: f(n) = g(n) + h(n), where g(s) is the total path cost to a node n, and h(n) is an admissible heuristic function that captures each nodes potential to reach the goal. The total path cost is calculated the same way as in the Uniform Cost Search algorithm, and the heuristic function follows the framework of Greedy search. An dictionary is used to store the sum of the terrain cost for each node in the path to the current node. This allows for the efficient calculation of a neighboring node cost and the maintenance of the least cost path to a node, which allows the algorithm to use the most cost-effective route to reach a node on the frontier. The use of a combined path cost (with lowest-cost dictionary) and heuristic function prioritizes movements that steer the search in the direction of the goal, avoiding fold, loops and high-cost terrains.*

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.


In [17]:
def a_star_search(world: List[List[str]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int], moves: List[Tuple[int, int]], heuristic: Callable) -> List[Tuple[int, int]]:
    pq: List[Tuple[Tuple[int, int], float]] = []  # frontier
    explored: Set[Tuple[int, int]] = set()  # explored list
    parents: Dict[Tuple[int, int], Tuple[int, int]] = {}  # child -> parent map
    path_costs: Dict[Tuple[int, int], float] = {start: 0.0}  # node -> shortest path cost to node (dont need to store heuristic bc it is path independent)

    priority_enqueue(pq, start, 0)

    while pq:  # Check if more neighbors to traverse
        node: Tuple[int, int] = priority_dequeue(pq)

        if node == goal:  # dijkstra style backtrack
            actions = []
            while node != start:
                parent = parents[node]
                actions.append((node[0] - parent[0], node[1] - parent[1]))  # calc offset
                node = parent
            return actions[::-1]  # switch order from child->parent to parent->child

        for neighbor in get_neighbors(world, node, moves):  # add neighbors to frontier
            nx, ny = neighbor
            if neighbor not in explored:
                cost = path_costs[node] + costs[world[ny][nx]]  # neighbors cost

                # if this is the first time we have seen neighbor or the cost to take this route is better than another way to get to neighbor
                if neighbor not in path_costs or cost < path_costs[neighbor]:
                    priority: float = cost + heuristic(neighbor, goal)
                    priority_enqueue(pq, neighbor, priority)

                    # store for later
                    parents[neighbor] = node
                    path_costs[neighbor] = cost

        explored.add(node)

    return []

<a id="pretty_print_path"></a>
## pretty_print_path

*`pretty_print_path` Given a terrain map, a starting and ending position, and a set of moves to make, display an html grid of the map with arrows for the path taken. Additionally, calculate the total path cost using a table mapping each terrain to a travel cost. This function does not modify the inputs.*

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** Int - The path cost.

In [18]:
def pretty_print_path(world: List[List[str]], path: List[Tuple[int, int]], start: Tuple[int, int], goal: Tuple[int, int], costs: Dict[str, int]) -> int:
    total_cost = 0

    x = start[0]
    y = start[1]

    map = deepcopy(world)  # dont modify world

    map[goal[1]][goal[0]] = "🎁"

    for dx, dy in path:
        node = (x + dx, y + dy)
        total_cost += costs[world[y][x]]
        
        if (dx, dy) == (0, 1):
            tile = "⏬"

        elif (dx, dy) == (1, 0):
            tile = "⏩"

        elif (dx, dy) == (-1, 0):
            tile = "⏪"

        elif (dx, dy) == (0, -1):
            tile = "⏫"

        else:
            tile = map[y][x]


        map[y][x] = tile
        x, y = node

    display_emoji_grid(map)
    return total_cost



<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

## Problems

## Problem 1. 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [19]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏬,🌲,🌲,🌲,🌲,🌲,🌲
⏩,⏩,⏩,⏩,⏩,⏩,⏬
🌲,🌲,🌲,🌲,🌲,🌲,⏬
🌲,🌲,🌲,🌲,🌲,🌲,⏬
🌲,🌲,🌲,🌲,🌲,🌲,🎁


total path cost: 12
[(0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [20]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏬,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏬,🌾,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,⏬,🌾,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,⏩,⏬,🌾,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,⏬,🌾,⛰,⛰,🌾
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,⏬,⛰,⛰,🌾,🌾
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⏩,⏩,⏬,🌾,🌾
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⏬,🌾,🌾
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,⏬,🌾,🌾
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⏩,⏩,⏬
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,⏬


total path cost: 98
[(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]


## Comments

(This is the place to leave me comments)

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.

## Extra: Testing different heuristics:


In [ ]:
# Testing different heuristics:

def manhattan_distance(start, goal):
    return abs(start[0] - goal[0]) + abs(start[1] - goal[1]) # |x1 - x2| + |y1 - y2|

    
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, manhattan_distance)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏩,⏬,🌾,⛰,⛰,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌾,🌾,🌾,🌾,🐊,🐊,🐊,🌾,🌾,🌾,🌾,🌾,🌾,🌾,🌾,⏬,⛰,🌋,🌋,⛰
🌾,🌾,🌾,🌾,🌾,⛰,⛰,⛰,🌾,🌾,🐊,🐊,🐊,🐊,🌾,🌾,🌋,🌾,🌾,🌾,⛰,🌾,⏬,⛰,⛰,🌋,🌾
🌾,🌾,🌾,🌾,⛰,⛰,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌾,🌋,🌋,🌋,🌋,🌾,⛰,🌾,⏩,⏬,⛰,🌋,🌾
🌾,🌾,🌋,⛰,⛰,🌋,🌋,⛰,⛰,🐊,🐊,🐊,🐊,🐊,🌋,🌋,🌲,🌋,🌋,🌋,⛰,⛰,🌾,⏩,⏩,⏩,⏬
🌲,🌾,🌋,🌋,🌋,🌋,⛰,⛰,⛰,🐊,🐊,🐊,🌾,🌾,🌾,🌋,🌲,🌲,🌋,🌋,⛰,⛰,🌾,⛰,⛰,🌾,⏬
🌲,🌾,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,🐊,🌾,🌾,🌾,🌾,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,⛰,🌾,🌾,⏬
🌲,🌲,🌲,🌋,🌲,⛰,🌾,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,⛰,🌾,🌾,⏬
🌲,🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,⛰,⛰,⛰,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,⛰,⛰,🌾,🌾,⏬
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,⛰,🌋,🌋,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌋,⛰,🌾,🌾,⏬
🌲,🌲,🌲,🌲,🌾,🌾,🌾,🌾,🌾,⛰,🌋,🌋,🌋,⛰,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌲,🌋,🌾,🌾,⛰,⏬


total path cost: 98
[(1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]
